In [15]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
def get_chatbot_response(client,model_name,messages,temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content
    
    return response

In [9]:
client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_CHATBOT_URL"),
)
model_name = os.getenv("MODEL_NAME")

## Get LLM response

In [9]:
messages = [{'role':'user','content':"What's the capital of Italy?"}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The capital of Italy is Rome (Italian: Roma).


## Prompt engineering

### Structured output

In [23]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
[
{
    "country": the country that you will get the capital of 
    "capital": the capital of the country stated
}
]
"""

messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':"What's the capital of Italy?"})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
{
    "country": "Italy",
    "capital": "Rome"
}
]


In [24]:
type(response)

str

In [25]:
response

'[\n{\n    "country": "Italy",\n    "capital": "Rome"\n}\n]'

In [26]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'}]

In [27]:
type(json_response)

list

## Input Structuring

In [29]:
user_prompt = """
Get me the capitals of the following countries:
```
1. Italy
2. France
3. Germany
``
"""
messages = [{'role':'system','content':system_prompt}]
messages.append({'role':'user','content':user_prompt})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "France",
        "capital": "Paris"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    }
]


In [30]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'France', 'capital': 'Paris'},
 {'country': 'Germany', 'capital': 'Berlin'}]

## Give the model time to think (Chain of thought)

In [31]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 4
}


In [32]:
259/2*8654+91072*33-12971

4113098.0

In [33]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 1343519
}


In [34]:
4113098.0-1343519

2769579.0

In [35]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one bellow. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to final result. Feel free to write here in free text. 
    result: The final number resulted from calculating the equation above
}
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)

In [36]:
print(response)

{
    "steps": "First, calculate the division: 259/2 = 129.5. Then, multiply 129.5 by 8654: 129.5 * 8654 = 112,141.5. Next, multiply 91072 by 33: 91072 * 33 = 3,003,336. Now, add 112,141.5 and 3,003,336: 112,141.5 + 3,003,336 = 4,115,477.5. Finally, subtract 12971 from 4,115,477.5: 4,115,477.5 - 12971 = 4,005,806.5.",
    "result": 4005806.5
}


In [37]:
4113098.0-4005806.5

107291.5

In [38]:
2769579.0/107291.5

25.813591943443797

## RAG - Retrieval Augmented Generation

### Asking about a subject that the LLM does not know anything about

In [39]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

There is no such thing as an "iPhone 16". Apple has not released an iPhone with that model number. The latest iPhone models available in the market are iPhone 13 series, which includes iPhone 13, iPhone 13 Mini, iPhone 13 Pro, and iPhone 13 Pro Max.

However, if you're looking for information on the latest rumors and leaks about the next-generation iPhone, I can provide you with some information.

Rumors suggest that the next-generation iPhone, likely to be called iPhone 14, will feature several upgrades and improvements, including:

1. Improved cameras: The iPhone 14 is expected to feature a new triple-camera setup with a wider-angle lens, a telephoto lens, and a ultra-wide-angle lens.
2. Faster processor: The iPhone 14 is expected to be powered by Apple's A15 Bionic chip, which will provide a significant boost in performance and efficiency.
3. Longer battery life: The iPhone 14 is expected to have a longer battery life, with some reports suggesting up to 12 hours of internet use on a

### Giving Context to the unknown subject

In [2]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [4]:
user_prompt = f"""
{iphone_16}

What's new in iphone 16?
"""

messages = [{'role':'user','content':user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

NameError: name 'get_chatbot_response' is not defined

### Automatically extract context data from database

In [3]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [4]:

data = [iphone_16,samsung_s23]

In [5]:

user_prompt = """What's new in iphone 16?"""

In [6]:
embedding_client = OpenAI(
        api_key=os.getenv("RUNPOD_TOKEN"), 
        base_url=os.getenv("RUNPOD_EMBEDDING_URL")
    )

In [7]:
def get_embedding(embedding_client,model_name,text_input):
    output = embedding_client.embeddings.create(input = text_input,model=model_name)
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings

In [10]:
user_prompt_embeddings = get_embedding(embedding_client,model_name,user_prompt)

In [11]:
user_prompt_embeddings

[[-0.05107622966170311,
  -0.03489547595381737,
  0.06362394988536835,
  -0.008907047100365162,
  -0.03724626451730728,
  -0.0420089028775692,
  0.015967046841979027,
  0.031155584380030632,
  -0.012799587100744247,
  0.04243631660938263,
  0.05556409806013107,
  0.013288062997162342,
  -0.07217226922512054,
  0.00139673484954983,
  0.09635180979967117,
  0.028697941452264786,
  0.11521918326616287,
  -0.14751963317394257,
  -0.07907198369503021,
  0.016165489330887794,
  -0.03666619956493378,
  -0.014616105705499649,
  -0.039841290563344955,
  -0.030025985091924667,
  0.030025985091924667,
  0.061517395079135895,
  0.0010771268280223012,
  -0.012753792107105255,
  -0.019157402217388153,
  -0.13848282396793365,
  -0.000405234401114285,
  -0.006754701491445303,
  0.01666923053562641,
  0.026835627853870392,
  0.00040475736022926867,
  -0.038497984409332275,
  -0.0031502859201282263,
  0.03492600470781326,
  -0.009632128290832043,
  0.06380712240934372,
  0.029629098251461983,
  0.091039

In [12]:
user_prompt_embeddings = user_prompt_embeddings[0]

In [13]:
user_prompt_embeddings

[-0.05107622966170311,
 -0.03489547595381737,
 0.06362394988536835,
 -0.008907047100365162,
 -0.03724626451730728,
 -0.0420089028775692,
 0.015967046841979027,
 0.031155584380030632,
 -0.012799587100744247,
 0.04243631660938263,
 0.05556409806013107,
 0.013288062997162342,
 -0.07217226922512054,
 0.00139673484954983,
 0.09635180979967117,
 0.028697941452264786,
 0.11521918326616287,
 -0.14751963317394257,
 -0.07907198369503021,
 0.016165489330887794,
 -0.03666619956493378,
 -0.014616105705499649,
 -0.039841290563344955,
 -0.030025985091924667,
 0.030025985091924667,
 0.061517395079135895,
 0.0010771268280223012,
 -0.012753792107105255,
 -0.019157402217388153,
 -0.13848282396793365,
 -0.000405234401114285,
 -0.006754701491445303,
 0.01666923053562641,
 0.026835627853870392,
 0.00040475736022926867,
 -0.038497984409332275,
 -0.0031502859201282263,
 0.03492600470781326,
 -0.009632128290832043,
 0.06380712240934372,
 0.029629098251461983,
 0.09103964269161224,
 -0.10141974687576294,
 -0.02

In [14]:
data_embeddings = [get_embedding(embedding_client,model_name,x)[0] for x in data]

In [18]:
data_embeddings

[[-0.0817207545042038,
  0.0334145724773407,
  0.031080130487680435,
  0.0008568206103518605,
  -0.04433915391564369,
  0.01893492415547371,
  0.03161415457725525,
  0.01858399622142315,
  0.04494946822524071,
  0.05462291091680527,
  0.07964569330215454,
  0.007201679516583681,
  -0.06286212056875229,
  -0.008788490667939186,
  0.031064871698617935,
  0.06408274173736572,
  0.13280385732650757,
  -0.16429594159126282,
  -0.0559045635163784,
  0.014395730569958687,
  0.05437878519296646,
  -0.04864185303449631,
  -0.008262096904218197,
  -0.01525016687810421,
  0.040341611951589584,
  0.04696349427103996,
  -0.006206108722835779,
  -0.010329527780413628,
  0.02720464952290058,
  -0.20958109200000763,
  0.008071374148130417,
  0.011389944702386856,
  0.04940474405884743,
  0.005126619245857,
  -0.0521816611289978,
  0.0013693871442228556,
  -0.02683846279978752,
  -0.001562970457598567,
  0.028837233781814575,
  0.04314904659986496,
  -0.00026868024724535644,
  0.06228232383728027,
  -0

In [19]:
len(data_embeddings)

2

In [16]:
data_similaraty_scores = cosine_similarity([user_prompt_embeddings], data_embeddings)

In [17]:
data_similaraty_scores

array([[0.7378098 , 0.52626918]])

In [20]:
closest_entry_index=data_similaraty_scores.argmax()
closest_entry_index

np.int64(0)

In [21]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [23]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
""" 

In [24]:
user_prompt_with_data

'\n\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine

In [27]:
messages = [{'role':'user','content':user_prompt_with_data}]
response = get_chatbot_response(client,model_name,messages)
print(response)

According to the text, the new features and updates in the iPhone 16 include:

1. Larger displays: 6.1-inch for the base model, 6.7-inch for the iPhone 16 Plus, 6.3 inches for the iPhone 16 Pro, and 6.9 inches for the iPhone 16 Pro Max.
2. Thinner bezels and a more durable Ceramic Shield.
3. New A18 chip (A18 Pro for the Pro models) with improved performance, neural engine capabilities, and faster GPU for gaming and machine learning tasks.
4. Upgraded camera systems:
	* Base iPhone 16: dual-camera setup with a 48MP main sensor.
	* Pro models: 48MP Ultra Wide and 5x telephoto camera, with Apple's "Camera Control" button for more flexible photography options.
5. Advanced audio features, including "Audio Mix" for refined audio capture during video recording.
6. Extended battery life, especially in the iPhone 16 Pro Max, which has the longest-lasting battery of any iPhone.
7. Switch to USB-C for faster charging and data transfer.
8. Pro models support up to 2x faster video encoding.
9. Sta